<a href="https://www.kaggle.com/code/jeffreyesedo/1st-ribo-note?scriptVersionId=148929773" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# Setting up an RNA Science Environment
!pip install arnie
!pip install draw_rna

# Install EternaFold
!conda config --set auto_update_conda false
!conda install -c bioconda eternafold --yes
# Manually setup EternaFold for Kaggle notebook
%env ETERNAFOLD_PATH=/opt/conda/bin/eternafold-bin
%env ETERNAFOLD_PARAMETERS=/opt/conda/lib/eternafold-lib/parameters/EternaFoldParams.v1

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# from arnie.mfe import mfe # Minimum Free Energy (mfe) function makes structure predictions about a given sequence
# from arnie.bpps import bpps # function that predicts the probability of every possible base pairing
# from draw_rna.ipynb_draw import draw_struct # plot RNA structures in 2D

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Datasets

In [ ]:
train= pd.read_csv("/kaggle/input/stanford-ribonanza-rna-folding/train_data.csv")

test= pd.read_csv("/kaggle/input/stanford-ribonanza-rna-folding/test_sequences.csv")

In [ ]:
print(f"Train dataset shape: {train.shape}\n")

print(f"Test dataset shape: {test.shape}")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print(f"Test Columns: {test.info()}")

In [ ]:
print(f"Train Columns: {train.info()}")

In [ ]:
# Count columns based on their Dtype
dtype_counts = train.dtypes.value_counts()
print(dtype_counts)

In [ ]:
# Columns with int dtype
numeric_columns = train.select_dtypes(include=['int64'])
numeric_columns.head()

In [ ]:
numeric_columns.info()

In [ ]:
numeric_columns["SN_filter"].value_counts()

In [ ]:
# Columns with Str dtype
str_columns = train.select_dtypes(include=['object'])
print(str_columns.columns)
str_columns.head()

In [ ]:
str_columns.info()

In [ ]:
# Uniqueness of the dataset name
str_columns['dataset_name'].value_counts()

In [ ]:
experiments_count= train["experiment_type"].value_counts()
print(experiments_count)

In [ ]:
float_columns = train.select_dtypes(include=['float64'])
float_columns.columns

In [ ]:
# Null values in reactivity and reactivity_error
float_columns.drop('signal_to_noise', axis=1).info(verbose=True, show_counts=True)


In [ ]:
print(float_columns["reactivity_error_0206"].isna().sum())
print(float_columns["reactivity_0083"].isna().sum())



In [ ]:
# Columns that are NaN
num_empty_cols= 0
cols_having_values=0


for col in float_columns.drop('signal_to_noise', axis=1):
    if float_columns[col].isna().sum() == 1643680:
        num_empty_cols+=1
    else:
        cols_having_values+=1
        
print(f"Number of Columns with only NaN values: {num_empty_cols} of 412 columns\n")
print(f"Number of Columns with values: {cols_having_values} of 412 columns")

In [ ]:
# print(len(train["sequence"][1]))
train.iloc[0:, 7:]